In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from pydantic import BaseModel, Field, TypeAdapter

In [3]:
import os
from langgraph.pregel.remote import RemoteGraph
from langgraph.types import Command as LGCommand

In [4]:
from imandra.u.agents.code_logician.graph import GraphState
from imandra.u.agents.code_logician.command import Command, InitStateCommand

In [6]:
# Connect to deployed agent in dev
from imandra import auth
from imandra.u.agents import create_thread_sync, get_remote_graph
agent = "code_logician"
# agent = "cogito"
# agent = "code_logician_python"
c = auth.Config()
url = c.get_url() + "/agents/" + agent
# url = "https://cogito.core-europe-west1.internal.dev.imandracapital.com/universe-agents/cogito"
graph = RemoteGraph(agent, url=url, headers=c.get_headers(), config={"configurable": {}})
create_thread_sync(graph)
url

'https://api.dev.imandracapital.com/v1beta1/agents/code_logician'

In [7]:
# url = "http://127.0.0.1:2024"
# graph = RemoteGraph(
#     "code_logician",
#     url=url,
# )
# thread = await graph.client.threads.create()
# config = {"configurable": {"thread_id": thread["thread_id"]}}

In [8]:
from imandra.u.agents.code_logician.graph import GraphState

In [17]:
gs = GraphState()
src_code = """def g(x: int) -> int:
    if x > 22:
        return 9
    else:
        return 100 + x

def f(x: int) -> int:
    if x > 99:
        return 100
    elif 70 > x > 23:
        return 89 + x
    elif x > 20:
        return g(x) + 20
    elif x > -2:
        return 103
    else:
        return 99"""

gs = gs.add_commands(
    [
        Command(type="init_state", src_code=src_code, src_lang="python"),
        Command(type="gen_formalization_data"),
        Command(type="gen_model"),
    ]
)
res = await gs.run(graph)
gs = res[0]

In [20]:
assert gs.last_fstate.status.name == "TRANSPARENT"


True

In [23]:
assert gs.steps[-1].meta.status == "done"

In [ ]:
# Verify the workflow completed successfully
assert gs.steps[-1]["meta"]["status"] == "done"

# Check that we have a formalization status
assert hasattr(gs.last_fstate, "status")
# Status should be either 'Transparent' or another valid status
assert gs.last_fstate.status is not None

In [15]:
assert gs.steps[-1]
assert gs.steps[-1].meta.status == "done"
fstate = gs.last_fstate
assert fstate.iml_code == "sadasd"

In [13]:
gs = GraphState()
src_code = """
def f(x):
    return x + 1
"""
gs = gs.add_commands([
    Command(type="init_state", src_code=src_code, src_lang="python"),
    Command(type="edit_state_element", update={"iml_code": "sadasd"})
])
res = await gs.run(graph)
gs = res[0]

In [24]:
gs = GraphState()
src_code = """def g(x: int) -> int:
    if x > 22:
        return 9
    else:
        return 100 + x

def f(x: int) -> int:
    if x > 99:
        return 100
    elif 70 > x > 23:
        return 89 + x
    elif x > 20:
        return g(x) + 20
    elif x > -2:
        return 103
    else:
        return 99"""
gs = GraphState()
gs = gs.add_commands([
    Command(type="init_state", src_code=src_code, src_lang="python"),
    Command(type="gen_formalization_data"),
    Command(type="gen_model"),
])
res = await gs.run(graph)
gs = res[0]
assert gs.last_fstate.status.name == "TRANSPARENT"
gs2 = gs.add_commands([
    Command(type="gen_region_decomps", function_name="f"),
    Command(type="gen_test_cases", decomp_idx=0),
])
res2 = await gs2.run(graph)
gs2 = res2[0]
assert gs2.steps[-1].meta.status == "done"
test_cases = gs2.last_fstate.region_decomps[0]["test_cases"]
assert len(test_cases['iml']) > 0
assert len(test_cases['src']) > 0